In [15]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.mllib.feature import StandardScaler,PCA

In [4]:
if __name__ == "__main__":
    sc = SparkSession .builder\
        .appName("PCA")\
        .getOrCreate()

In [5]:
df = sc.read.csv("Glass.data",header=True)

In [6]:
df.show()

+-----+-------+-----+----+----+-----+----+----+----+----+-----+
|index|     RI|   Na|  Mg|  Al|   Si|   K|  Ca|  Ba|  Fe|Class|
+-----+-------+-----+----+----+-----+----+----+----+----+-----+
|    1|1.52101|13.64|4.49|1.10|71.78|0.06|8.75|0.00|0.00|    1|
|    2|1.51761|13.89|3.60|1.36|72.73|0.48|7.83|0.00|0.00|    1|
|    3|1.51618|13.53|3.55|1.54|72.99|0.39|7.78|0.00|0.00|    1|
|    4|1.51766|13.21|3.69|1.29|72.61|0.57|8.22|0.00|0.00|    1|
|    5|1.51742|13.27|3.62|1.24|73.08|0.55|8.07|0.00|0.00|    1|
|    6|1.51596|12.79|3.61|1.62|72.97|0.64|8.07|0.00|0.26|    1|
|    7|1.51743|13.30|3.60|1.14|73.09|0.58|8.17|0.00|0.00|    1|
|    8|1.51756|13.15|3.61|1.05|73.24|0.57|8.24|0.00|0.00|    1|
|    9|1.51918|14.04|3.58|1.37|72.08|0.56|8.30|0.00|0.00|    1|
|   10|1.51755|13.00|3.60|1.36|72.99|0.57|8.40|0.00|0.11|    1|
|   11|1.51571|12.72|3.46|1.56|73.20|0.67|8.09|0.00|0.24|    1|
|   12|1.51763|12.80|3.66|1.27|73.01|0.60|8.56|0.00|0.00|    1|
|   13|1.51589|12.88|3.43|1.40|73.28|0.6

In [10]:
# we will dos tandard scaling before applying PCA

In [7]:
#we do not need to scale our label coulmn, so let's seperate it

label = df.select('Class')

In [8]:
features = df.drop('Class')

In [9]:
col_names = features.columns
features_rdd = features.rdd.map(lambda row: row[0:])

In [ ]:
features_rdd.collect()

In [11]:
scaler1 = StandardScaler().fit(features_rdd)

In [12]:
scaled_features=scaler1.transform(features_rdd)

In [ ]:
for data in scaled_features.collect():
    print(data)

In [17]:
pca = PCA(k=3)
model = pca.fit(scaled_features)

In [20]:
result = model.transform(scaled_features)

MapPartitionsRDD[41] at mapPartitions at PythonMLLibAPI.scala:1336

In [30]:
#store dense vector in a dataframe

In [31]:
df =result.map(lambda x: (x, )).toDF(["PCA_Features"])

In [32]:
df.show(truncate=False)

+------------------------------------------------------------+
|PCA_Features                                                |
+------------------------------------------------------------+
|[121.27139068803648,-255.7711564625892,-4.196845118886992]  |
|[120.05549997551647,-254.54587401309547,-4.381436698368739] |
|[119.84798719645046,-254.20344475546793,-4.381251736462124] |
|[120.46108996886237,-254.67949602807138,-3.9459383020114496]|
|[120.35044011370479,-254.4653916159397,-4.314355448974061]  |
|[120.45089628391106,-254.3745775769602,-2.96892323108168]   |
|[120.42061239242885,-254.5173681539074,-4.370326737605008]  |
|[120.5600319131514,-254.53042059230148,-4.455718059263219]  |
|[120.21596832653775,-255.23024547077227,-3.9323571038112295]|
|[120.53837763905427,-254.73937060667092,-3.712336421811534] |
|[120.36864286958993,-254.29818100371133,-3.148700608085193] |
|[120.55406975213215,-254.6982022163838,-3.9768907100498794] |
|[120.42918755933822,-254.3202896086675,-3.352150808310

In [33]:
sc.stop()